### Car Model Verification

https://ods.ai/competitions/mcs_car_verification

https://github.com/AlexanderParkin/MCS2022.Baseline

In [1]:
import os

import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import models
from torchvision import transforms

from pytorch_metric_learning import distances, losses, miners, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

### Get data

In [2]:
data_path = './data/image'
data = []
for path, folder, files in os.walk(data_path):
    if files:
        for f in files:
            data.append(((path + '/' + f), f[:-4]))

In [3]:
data = sorted(data, key=lambda x: x[1])

In [4]:
img_path, img_ids = list(zip(*data))

In [5]:
df_img = pd.DataFrame()
df_img['id'] = img_ids
df_img['img_path'] = img_path

In [6]:
labels_path = './data/label'
labels = []
for path, folder, files in os.walk(labels_path):
    if files:
        for f in files:
            labels.append(((path + '/' + f), f[:-4]))

In [7]:
labels = sorted(labels, key=lambda x: x[1])

### Create dataset

In [8]:
lbs_path, lbs_id = list(zip(*labels))

In [9]:
df_lbs = pd.DataFrame()
df_lbs['id'] = lbs_id
df_lbs['lbs_path'] = lbs_path

In [10]:
df = pd.merge(df_img, df_lbs, how='inner', on = 'id')

In [11]:
labels = df['lbs_path']

In [12]:
bbs = []
for path in tqdm(labels):
    with open(path) as f:
        lines = f.readlines()
    bbs.append(lines[-1][:-1].split(' '))

100%|██████████| 136894/136894 [00:41<00:00, 3321.28it/s]


In [13]:
df['bb'] = bbs

In [14]:
lbs = [lb.split('/')[3] for lb in labels]

In [15]:
df['label'] = lbs

In [16]:
class CustomDataset(Dataset):
    
    def __init__(self, df):
        self.images = df['img_path'].values
        self.bbs = df['bb'].values
        self.labels = df['label'].values
        
    def __getitem__(self, index):
        image_path = self.images[index]
        try:
            image = Image.open(image_path)
        except:
            image = Image.open(self.images[0])
            print('OPEN FILE ERR', image_path)
        left, top, right, bottom = self.bbs[index]
        try:
            image = image.crop((int(left), int(top), int(right), int(bottom)))
        except:
            print('BB ERRR', image_path)
        image = image.resize((224, 224))
        image = transforms.ToTensor()(image)
        try:
            image = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(image)
        except:
            image = torch.zeros([3, 224, 224])
            print('IMAGE FORMAT ERRR', image_path)
        if image.shape != torch.Size([3, 224, 224]):
            image = torch.zeros([3, 224, 224])
            print('IMG ERR', image_path)
        label = int(self.labels[index]) - 1
        return image, label
        
    def __len__ (self):
        return len(self.images)

In [17]:
train, test = train_test_split(df, test_size=0.05)

In [18]:
dataset_train = CustomDataset(train[:100])
dataset_test = CustomDataset(test[:10])

In [19]:
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=2, shuffle=True)

### Model

In [20]:
model = models.resnet18(pretrained=False)

In [21]:
n_classes = len(list(set(df['label'].values)))

In [22]:
num_ftrs = model.fc.in_features

In [23]:
model.fc = nn.Linear(num_ftrs, n_classes)

In [24]:
model.load_state_dict(torch.load('./models/resnet18.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [25]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [26]:
model.to(device);

In [27]:
optimizer = torch.optim.AdamW(model.parameters())

### Train loop

In [28]:
def train_nn(model, data_loader):
    loss_fn = torch.nn.CrossEntropyLoss()
    model.train()
    for data in tqdm(data_loader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

In [29]:
def eval_nn(model, data_loader):
    predicted = []
    labels = []
    model.eval()
    with torch.no_grad():
        for data in tqdm(data_loader):
            x, y = data
            x = x.to(device)

            outputs = model(x)
            _, predict = torch.max(outputs.data, 1)
            predict = predict.cpu().detach().numpy().tolist()
            predicted += predict
            labels += y
        score = f1_score(labels, predicted, average='macro')
        print(score)
    return labels, predicted, score

In [30]:
train_nn(model, train_loader)

100%|██████████| 1/1 [00:16<00:00, 16.19s/it]


In [31]:
_, _, score = eval_nn(model, test_loader)

100%|██████████| 5/5 [00:00<00:00,  7.57it/s]

0.10714285714285714


In [32]:
torch.save(model.state_dict(), './models/model_' + str(round(score, 3)) + '.pth')

### Metric learning

In [33]:
model.fc = torch.nn.Identity()

In [34]:
distance = distances.CosineSimilarity()
reducer = reducers.ThresholdReducer(low=0)
loss_func = losses.TripletMarginLoss(margin=0.2, distance=distance, reducer=reducer)
mining_func = miners.TripletMarginMiner(
    margin=0.2, distance=distance, type_of_triplets="semihard"
)

In [35]:
model.to(device);

In [36]:
model.train()
for epoch in range(1):
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = model(inputs)
        indices_tuple = mining_func(embeddings, labels)
        loss = loss_func(embeddings, labels, indices_tuple)
        loss.backward()
        optimizer.step()

100%|██████████| 1/1 [00:16<00:00, 16.07s/it]


In [37]:
torch.save(model.state_dict(), './models/model_mc.pth')